In [3]:
%%file Constant.py
import math

# This class contains all the constants used in this project, ranging from mathematical to predefined constants
class Constant:
    
    EPS = 0.001
    PI = math.acos(-1)
    EAST_TO_NORTH_ANGLE = 90
    DEGREE_TO_RADIAN_RATIO = 180 / PI

Overwriting Constant.py


In [4]:
%%file GraphUtility.py
# This class contains utility static methods related to 2D graphs in general
class GraphUtility:
    
    @staticmethod
    def calculate_manhattan_distance(x1, y1, x2, y2):
        return abs(x1 - x2) + abs(y1 - y2)
    
    @staticmethod
    def calculate_squared_distance(x1, y1, x2, y2):
        return (x1 - x2) * (x1 - x2) + (y1 - y2) * (y1 - y2)
    
    @staticmethod
    def calculate_euclidean_distance(x1, y1, x2, y2):
        return math.sqrt(GraphUtility.calculate_squared_distance(x1, y1, x2, y2))

Overwriting GraphUtility.py


In [5]:
%%file Node.py
import math

# This class represents a graph node, its properties, and utility methods
class Node:
    
    def __init__(self, node, x, y, name):
        self.id = node
        self.x = x
        self.y = y
        self.name = name
        self.adj = {}
    
    def calculate_vector_to(self, neighbour):
        return (neighbour.get_x() - self.x, neighbour.get_y() - self.y)
    
    def calculate_manhattan_distance_to(self, neighbour):
        return GraphUtility.calculate_manhattan_distance(self.x, self.y, neighbour.get_x(), neighbour.get_y())
    
    def calculate_euclidean_distance_to(self, neighbour):
        return GraphUtility.calculate_euclidean_distance(self.x, self.y, neighbour.get_x(), neighbour.get_y())
    
    def calculate_euclidean_distance_from_point(self, x, y):
        return GraphUtility.calculate_euclidean_distance(x, y, self.x, self.y)
    
    def add_neighbour(self, neighbour):
        self.adj[neighbour] = self.calculate_vector_to(neighbour)
    
    def get_id(self):
        return self.id
    
    def get_x(self):
        return self.x
    
    def get_y(self):
        return self.y
    
    def get_name(self):
        return self.name
    
    def get_degree(self):
        return len(self.adj)
    
    def get_neighbours(self):
        return self.adj
    
    # This method returns the rotation difference from a given point with a given heading angle (in degrees relative to North)
    def get_rotation_difference_from_point(self, x, y, heading_angle, north_angle):
        target_rotation_relative_to_east = math.atan2(self.y - y, self.x - x) * -Constant.DEGREE_TO_RADIAN_RATIO
        target_rotation_relative_to_north = (target_rotation_relative_to_east + Constant.EAST_TO_NORTH_ANGLE - 
                                             heading_angle - north_angle)
        # Normalize the target rotation angle to (-180, 180]
        if (target_rotation_relative_to_north <= -180):
            target_rotation_relative_to_north += 360
        if (target_rotation_relative_to_north > 180):
            target_rotation_relative_to_north -= 360
            
        return target_rotation_relative_to_north
    
    def __repr__(self):
        return ("Node id: " + str(self.id) + ", "
                "Node name: " + self.name + ", "
                "Adjacent to: " + str([x.id for x in self.adj]))
    
    def __str__(self):
        return self.__repr__()

Overwriting Node.py


In [6]:
%%file Edge.py
# This class represents a graph edge, its properties, and utility methods
class Edge:
    
    def __init__(self, node_one, node_two):
        self.node_one = node_one
        self.node_two = node_two
        self.manhattan_distance = node_one.calculate_manhattan_distance_to(node_two)
        self.euclidean_distance = node_one.calculate_euclidean_distance_to(node_two)
        
    def get_either(self):
        return self.node_one
    
    def get_other(self, node):
        if (node.get_id() == self.node_one.get_id()):
            return self.node_two
        else:
            return self.node_one
    
    def get_manhattan_distance(self):
        return self.manhattan_distance
    
    def get_euclidean_distance(self):
        return self.euclidean_distance
    
    # This method returns the normal length from a point to this edge in cartesian coordinate
    def get_normal_length_from_point(self, x, y):
        node_one = self.get_either()
        node_two = self.get_other(node_one)
        
        node_one_to_node_two_vector = np.array([
                node_two.get_x() - node_one.get_x(), 
                node_two.get_y() - node_one.get_y()
            ])
        node_one_to_point_vector = np.array([
                x - node_one.get_x(), 
                y - node_one.get_y()
            ])
        normal_length = (np.linalg.norm(np.cross(node_one_to_node_two_vector, node_one_to_point_vector)) / 
                         np.linalg.norm(node_one_to_node_two_vector))
        
        return normal_length
    
    def get_distance_from_first_node_to_normal_of_point(self, x, y):
        node_one = self.get_either()
        node_two = self.get_other(node_one)
        
        node_one_to_node_two_vector = np.array([
                node_two.get_x() - node_one.get_x(), 
                node_two.get_y() - node_one.get_y()
            ])
        node_one_to_point_vector = np.array([
                x - node_one.get_x(), 
                y - node_one.get_y()
            ])
        distance_from_first_node_to_normal = (np.dot(node_one_to_node_two_vector, node_one_to_point_vector) / 
                                              np.linalg.norm(node_one_to_node_two_vector))
        
        return distance_from_first_node_to_normal
    
    # This method returns the distance to the nearest node in this edge from a given point
    def get_distance_to_nearest_node_from_point(self, x, y):
        node_one = self.get_either()
        node_two = self.get_other(node_one)
        
        return min(
            GraphUtility.calculate_euclidean_distance(x, y, node_one.get_x(), node_one.get_y()), 
            GraphUtility.calculate_euclidean_distance(x, y, node_two.get_x(), node_two.get_y())
        )
    
    # This method returns the nearest node in this edge from a given point
    def get_nearest_node_from_point(self, x, y):
        node_one = self.get_either()
        node_two = self.get_other(node_one)
        
        dist_to_node_one = GraphUtility.calculate_euclidean_distance(x, y, node_one.get_x(), node_one.get_y())
        dist_to_node_two = GraphUtility.calculate_euclidean_distance(x, y, node_two.get_x(), node_two.get_y())
        
        if (dist_to_node_one < dist_to_node_two):
            return node_one
        else:
            return node_two
    
    def __repr__(self):
        return ("Edge consists of node " + str(self.node_one.get_id()) + " (" + self.node_one.get_name() + ")"
                " and node " + str(self.node_two.get_id()) + " (" + self.node_two.get_name() + ")"
                ", distance between the two nodes is " + str(self.euclidean_distance))
    
    def __str__(self):
        return self.__repr__()

Overwriting Edge.py


In [7]:
%%file Graph.py
import math
import numpy as np

# This class represents an undirected (bidirectional) graph with adjacency list approach in storing the neighbouring nodes
class Graph:
    
    def __init__(self, json = None):
        self.node_map = {}
        self.num_nodes = 0
        self.edges = []
        self.north_angle = None
        
        if (json is not None):
            self.north_angle = float(json["info"]["northAt"])
            for node in json["map"]:
                node_id = int(node["nodeId"])
                x = int(node["x"])
                y = int(node["y"])
                node_name = node["nodeName"]
                link_to = node["linkTo"].split(", ")

                self.add_node(node_id, x, y, node_name)

                for i in range(len(link_to)):
                    self.add_edge(node_id, int(link_to[i]))
                    
            print("Graph is created successfully from the given JSON\n")
            print(self)
    
    def add_node(self, node_id, x, y, node_name):
        self.num_nodes = self.num_nodes + 1
        new_node = Node(node_id, x, y, node_name)
        self.node_map[node_id] = new_node
        return new_node
    
    def is_valid_node_id(self, node_id):
        if (node_id not in self.node_map):
            return False
        return True
    
    def add_edge(self, node_one_id, node_two_id):
        if (self.is_valid_node_id(node_one_id) and self.is_valid_node_id(node_two_id)):
            self.node_map[node_one_id].add_neighbour(self.node_map[node_two_id])
            self.node_map[node_two_id].add_neighbour(self.node_map[node_one_id])
            # Store the newly created edge
            new_edge = Edge(
                self.node_map[node_one_id], 
                self.node_map[node_two_id]
            )
            self.edges.append(new_edge)
    
    def get_node(self, node):
        return self.node_map[node]
    
    def get_degree(self, node):
        return self.node_map[node].get_degree()
    
    def get_num_nodes(self):
        return self.num_nodes
    
    def get_nodes(self):
        return self.node_map.values()
    
    def get_edges(self):
        return self.edges
    
    def get_north_angle(self):
        return self.north_angle
    
    # This method returns the nearest edge from a point in cartesian coordinate
    def get_nearest_edge_from_point(self, x, y):
        nearest_edge = None
        shortest_normal_length = math.inf
        shortest_distance_to_nearest_node_in_edge = math.inf
        
        for edge in self.edges:
            normal_length = edge.get_normal_length_from_point(x, y)
            # Case 1: If normal length is longer than the shortest so far, ignore this edge
            if (normal_length > shortest_normal_length):
                continue
            # Case 2: If normal length is shortest so far, check further
            distance_from_first_node_to_normal_of_point = edge.get_distance_from_first_node_to_normal_of_point(x, y)
            # Case 1.1: If projection point lies inside the edge, update nearest edge
            if (distance_from_first_node_to_normal_of_point >= 0 and 
                distance_from_first_node_to_normal_of_point <= edge.get_euclidean_distance()):
                # Update nearest edge
                nearest_edge = edge
                shortest_normal_length = normal_length
                shortest_distance_to_nearest_node_in_edge = edge.get_distance_to_nearest_node_from_point(x, y)
            # Case 1.2: If projection point lies outside the edge, check further
            else:
                distance_to_nearest_node_in_edge = edge.get_distance_to_nearest_node_from_point(x, y)
                # Case 1.2.1: If distance to nearest node is smaller, update nearest edge
                if (distance_to_nearest_node_in_edge < shortest_distance_to_nearest_node_in_edge):
                    # Update nearest edge
                    nearest_edge = edge
                    shortest_distance_to_nearest_node_in_edge = distance_to_nearest_node_in_edge
                    
        return nearest_edge
    
    def __repr__(self):
        result = "Nodes in this graph:\n"
        for node in self.node_map.values():
            result += str(node) + "\n"
        result += "North is at " + str(self.north_angle) + " degrees"
        return result
    
    def __str__(self):
        return self.__repr__()

Overwriting Graph.py


In [8]:
%%file Dijkstra.py
import heapq

# This class implements the Dijkstra algorithm and various utility methods related to it
class Dijkstra:
    
    def __init__(self, graph, source):
        num_nodes = graph.get_num_nodes()
        self.source_id = source
        self.dist_to = {}  # distance of shortest s->v path
        self.prev = {}  # previous node on shortest s->v path
        self.pq = [] # priority queue of vertices
        
        nodes = graph.get_nodes()
        
        for node in nodes:
            node_id = node.get_id()
            self.dist_to[node_id] = math.inf
            self.prev[node_id] = None
        self.dist_to[source] = 0.0
        
        # relax vertices in order of distance from s
        heapq.heappush(self.pq, (source, self.dist_to[source]))
        while(len(self.pq)):
            v = heapq.heappop(self.pq)[0]
            for neighbour in graph.get_node(v).get_neighbours():
                weight = graph.get_node(v).calculate_manhattan_distance_to(neighbour)
                self.__relax(v, neighbour.get_id(), weight)
                
    def __relax(self, v_id, w_id, weight):            
        # get the index, since we start from 1, we need to minus one
        if (self.dist_to[w_id] > self.dist_to[v_id] + weight):
            self.dist_to[w_id] = self.dist_to[v_id] + weight
            self.prev[w_id] = v_id
            heapq.heappush(self.pq, (w_id, self.dist_to[w_id]))
                
    def dist_to_node(self, v):
        return self.dist_to[v]
    
    def get_path(self, target_id):
        path = []
        current_node_id = target_id
        while (current_node_id != self.source_id):
            path.append(current_node_id)
            current_node_id = self.prev[current_node_id]
        path.append(current_node_id)
        return path[::-1]

Overwriting Dijkstra.py


In [9]:
%%file TestPathFinding.py
import requests
import json

r = requests.get("http://showmyway.comp.nus.edu.sg/getMapInfo.php?Building=COM1&Level=2")

g = Graph(r.json())

source = int(input("Enter source node id: "))
shortest_path = Dijkstra(g, source)
print()

destination = int(input("Enter destination node id: "))
print()

distance = shortest_path.dist_to_node(destination)

print("Shortest path from " + str(source) + " to " + str(destination) + ": ")
print(shortest_path.get_path(destination))
print()

print("Shortest distance is " + str(distance / 100) + "m")
print()

while (True):
    x = int(input("Enter current x-coordinate: "))
    y = int(input("Enter current y-coordinate: "))
    heading_angle = int(input("Enter angular difference from the North direction (in degrees): "))
    print()

    nearest_edge = g.get_nearest_edge_from_point(x, y)
    nearest_node = nearest_edge.get_nearest_node_from_point(x, y)
    rotate_direction = nearest_node.get_rotation_difference_from_point(x, y, heading_angle, g.get_north_angle())
    walk_distance = nearest_node.calculate_euclidean_distance_from_point(x, y)

    print("The nearest edge is: ")
    print(nearest_edge)
    print()

    print("The nearest node is: ")
    print(nearest_node)
    print()

    print("To get to the nearest node, you have to: ")
    print("Rotate " + str(rotate_direction) + " degrees and walk " + str(walk_distance / 100) + " meters")

Overwriting TestPathFinding.py
